In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from experiment_train import *
from experiment_datasets import *
from helpers import *

Using TensorFlow backend.


Initialized TensorFlow


In [36]:
# network configuration
N = [100, 50]

def get_p_arr(p):
    """ p array with failure on the first layer """
    return [0, p]  + [0] * len(N)

# prob. of failure at inference
p_inference = get_p_arr(0.1)

# prob. of failure at train
p_train = get_p_arr(0.00)

# Lips. coeff
KLips = 1.

# activation function
activation = 'relu'

# number of repeats for error estimation
repetitions = 5000

In [ ]:
# for some reason, running regression results in lower accuracy with dropout at inference than without
exp = MNISTExperiment(N = N, p_inference = p_inference, p_train = p_train, KLips = KLips, epochs = 50,
                              activation = activation, reg_type = None, reg_coeff = 0, do_print = 'plot',
                              name = 'experiment_dropout_first', seed = 0, batch_size = 10000)

In [34]:
np.argmax(exp.predict_correct(exp.x_train[:15]), axis = 1)

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 4, 7, 7, 7])

In [33]:
np.argmax(exp.y_train[:15], axis = 1)

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1])

In [21]:
exp.get_accuracy_correct()

{'train': 0.09245, 'test': 0.094}

In [10]:
exp.predict_correct(exp.x_train[:15]).shape

(15, 10)

In [46]:
exp.get_mae_correct()

{'train': 201.20902472014473, 'test': 206.23956682167804}

In [55]:
self = exp
repetitions = 100

In [51]:
(self.predict_correct(self.x_train) - self.y_train).shape

(404, 1)

11.687158131892136

In [30]:

np.random.randn()

1.3620400860402924

In [13]:
argmax_accuracy(np.array([[0,0,1], [1, 0, 0], [0, 1, 0]]), np.array([[0,2,1], [1, 0, 0], [0, 1, 0]]))

0.6666666666666666

In [15]:
self.get_accuracy_correct()

NameError: name 'self' is not defined

In [3]:
matrix_argmax(np.array([[1,2,3],[4,5,6]]))

(1, 2)

In [26]:
model = create_fc_crashing_model(exp.N, exp.W, exp.B, [0] * len(exp.N))

Initialized TensorFlow
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 3)                 42        
_________________________________________________________________
dense_74 (Dense)             (None, 1)                 4         
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.get_weights()[0]

array([[ 0.6933325 ,  0.15554036,  0.38530684],
       [ 0.87895066,  0.7311023 , -0.3819057 ],
       [ 0.37406954, -0.06078139, -0.03907155],
       [ 0.16104998,  0.05508433,  0.57182723],
       [ 0.29991776,  0.04631066,  0.17551169],
       [ 0.13229205,  0.5846118 , -0.07905547],
       [ 0.12420947, -0.33641812, -0.9999507 ],
       [ 0.25778452,  0.33764556, -0.28968686],
       [ 0.89168525, -0.5718634 ,  0.01936218],
       [-0.07200533,  0.5997912 ,  0.5777442 ],
       [ 0.06218959,  0.14691333, -0.34680402],
       [-0.77551806, -0.13787659,  0.06273933],
       [ 0.48397392,  0.47019798, -0.15050787]], dtype=float32)

In [28]:
exp.W[0]

array([[ 0.69333249,  0.87895066,  0.37406954,  0.16104998,  0.29991776,
         0.13229205,  0.12420947,  0.25778452,  0.89168525, -0.07200533,
         0.06218959, -0.77551806,  0.48397392],
       [ 0.15554036,  0.73110229, -0.06078139,  0.05508433,  0.04631066,
         0.58461177, -0.33641812,  0.33764556, -0.57186341,  0.59979123,
         0.14691333, -0.13787659,  0.47019798],
       [ 0.38530684, -0.3819057 , -0.03907155,  0.57182723,  0.17551169,
        -0.07905547, -0.99995071, -0.28968686,  0.01936218,  0.57774419,
        -0.34680402,  0.06273933, -0.15050787]])